In [1]:
import networkx as nx
import random
import numpy as np
import pandas as pd
from network_build import *

In [2]:
driver = pd.read_csv("..//..//..//Database//NYC_trip//driver.csv")
order = pd.read_csv("..//..//..//Database//NYC_trip//order.csv")
# area = np.load("Database//NYC_area//NY_area.npy")

area = pd.read_csv("..//..//..//Database//NYC_area//NY_area_df.csv", index_col=0)
area.columns = [int(float(s)) for s in area.columns]
area.index = [int(float(s)) for s in area.index]
order["call_time"] = pd.to_datetime(order["call_time"])
order["end_time"] = pd.to_datetime(order["end_time"])
start_time = pd.to_datetime("2022-06-01 06:00:00 AM")
end_time = pd.to_datetime("2022-06-01 06:30:00 AM")
order_pick = order[(order["call_time"] > start_time) & (order["call_time"] <= end_time)]
order_pick = order_pick[["sid", "call_time", "eid", "end_time"]]
order_pick = order_pick.values
driver["time"] = pd.date_range(start=start_time, end=start_time, periods=len(driver))
driver_pick = driver[["id", "time"]]
driver_pick = driver_pick.values
print(len(order_pick))

898


In [3]:
net = ConstructNetwork(order_pick, driver_pick, area, seed = 10, void = 10)
G = net.getConnectivity()
print(len(G.edges))

range(0, 898)
62012


In [5]:
a = nx.degree_centrality(G)
b = nx.betweenness_centrality(G)
c = nx.closeness_centrality(G)
d = nx.eigenvector_centrality(G)

df = pd.DataFrame.from_dict([a,b,c]).T
df.columns = ['degree_centrality', 'betweenness_centrality', 'closeness_centrality']
df

,degree_centrality,betweenness_centrality,closeness_centrality
s,0.357526,0.000000e+00,0.000000
t,0.678584,0.000000e+00,0.756764
dr0,0.005005,8.526266e-08,0.000358
dr1,0.030032,8.915449e-07,0.000358
dr2,0.000715,1.278705e-10,0.000358
...,...,...,...
td895,0.051484,3.767782e-04,0.016982
to896,0.020021,8.835852e-05,0.078165
td896,0.000715,1.278705e-07,0.059529
to897,0.012156,7.520750e-05,0.038780
